# Aviso
Não conseguimos refinar esse notebook para a entrega da sprint 3 e está planejado para ser analisado<br>
se realmente vamos realizar a análise preditiva dividida por níveis de valor de sinistro.<br>
O grupo decidirá na planning da sprint 4.

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs #???
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import seaborn as sns

pd.set_option('display.expand_frame_repr', False)

df = pd.read_csv('df_completo_codificado(10).csv')

## Checagem dos _imports_ do banco

In [ ]:
df.info()

In [ ]:
df

## Separação por níveis

In [ ]:
faixas_valor = [0, 50, 100, 200, 500, 10000, float("inf")]

categorias_valor = [1,2,3,4,5,6]

df["Categoria Sinistro"] = pd.cut(df["Valor Pago Sinistro"], bins=faixas_valor, labels=categorias_valor)

df['Categoria Sinistro'].value_counts()

## Verificação dos valores nos níveis

In [ ]:
# Definir variáveis diretamente
for nivel in range(1, 7):
    exec(f'df_nivel{nivel} = df.loc[df["Categoria Sinistro"] == {nivel}]')

df_nivel1.max()

## Plot do _Elbow Method_ para cada nível

## Cálculo do k ótimo (Elbow)

In [ ]:
# Fonte: https://medium.com/pizzadedados/kmeans-e-metodo-do-cotovelo-94ded9fdf3a9

x1, y1 = 2, wcss[0]
x2, y2 = 20, wcss[len(wcss)-1]

distances = []
for i in range(len(wcss)):
    x0 = i+2
    y0 = wcss[i]
    numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
    denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    distances.append(numerator/denominator)

distances.index(max(distances)) + 2

In [ ]:
# Definir o intervalo de k_values
k_values = range(1, 9)

# Criar a figura e os subplots
fig, axs = plt.subplots(2, 3, figsize=(18, 10))  # Ajuste o tamanho da figura conforme necessário
fig.suptitle('Método Elbow para Encontrar o k Ideal por Nível', fontsize=20, fontweight='bold')

# Iterar sobre cada nível e plotar em subplots
for nivel in range(1, 7):
    # Filtrar o DataFrame para o nível atual
    df_nivel = df.loc[df['Categoria Sinistro'] == nivel]
    
    wcss = []

    # Calcular WCSS para cada valor de k
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(df_nivel)
        wcss.append(kmeans.inertia_)

    # Encontrar o k ótimo usando o método da distância
    x1, y1 = 2, wcss[0]
    x2, y2 = 8, wcss[-1]  # 8 é o último valor de k
    
    distances = []
    for i in range(len(wcss)):
        x0 = i + 1  # k_values começa em 1, mas o índice começa em 0
        y0 = wcss[i]
        numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominator = np.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)
        distances.append(numerator / denominator)
    
    k_otimo = distances.index(max(distances)) + 1  # Adiciona 1 porque o índice começa em 0

    # Escolher o subplot para o nível atual
    ax = axs[(nivel - 1) // 3, (nivel - 1) % 3]

    # Escolher uma paleta de cores
    viridis_palette = sns.color_palette("viridis", n_colors=6)
    
    # Plotar os dados com Seaborn
    sns.lineplot(x=k_values, y=wcss, marker='o', linewidth=2, color=viridis_palette[nivel - 1], ax=ax)

    
    # Destacar o k ótimo no gráfico
    ax.plot(k_otimo, wcss[k_otimo - 1], 'ro', markersize=10)  # Ponto ótimo em vermelho
    ax.annotate(f'k={k_otimo}', (k_otimo, wcss[k_otimo - 1]), textcoords="offset points", xytext=(10,15), ha='center', fontsize=12, color='red')
    
    # Configurar os rótulos e título
    ax.set_xlabel('Número de Clusters (k)', fontsize=12)
    ax.set_ylabel('WCSS', fontsize=12)
    ax.set_title(f'Nível {nivel}', fontsize=14)
    
    # Remover a grade
    ax.grid(False)

    # Print do valor ótimo de k
    print(f'Nível {nivel}: k = {k_otimo}')

# Ajustar o layout para que não haja sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta o título da figura
plt.show()

## VarianceThreshold

In [ ]:
# Definir o limiar de variância para a seleção de características
var_thr = VarianceThreshold(threshold=0.25)  # Ajusta o threshold conforme necessário

# Ajustar o VarianceThreshold aos dados
var_thr.fit(df)

# Obter o suporte (máscara booleana) para as características selecionadas
support = var_thr.get_support()

# Identificar as colunas constantes e quase constantes
con_cols = [col for col in df.columns if col not in df.columns[support]]

# Remover as colunas constantes e quase constantes do DataFrame
df_reduzido = df.drop(con_cols, axis=1)

# Exibir as colunas removidas e o DataFrame reduzido
print("Colunas removidas:", con_cols)
print("DataFrame reduzido:")
print(df_reduzido.head())

## StandardScaler

In [ ]:
# Inicializar o StandardScaler
scaler = StandardScaler()

# Ajustar o scaler aos dados e transformar os dados
scaled_data = pd.DataFrame(scaler.fit_transform(df_reduzido), columns=df_reduzido.columns)

# Exibir as primeiras linhas dos dados normalizados
print("Dados normalizados:")
print(scaled_data.head())

## PCA

In [ ]:
# Inicializar o PCA com o número de componentes igual ao número de características normalizadas
pca = PCA(n_components=scaled_data.shape[1])

# Ajustar o PCA aos dados normalizados
pca.fit(scaled_data)

# Calcular a variância explicada e a variância explicada acumulada
explained_variance = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance)

# Função para plotar a variância explicada
def pca_plot(cumulative_explained_variance, data): # data ???
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
    plt.xlabel('Número de Componentes Principais')
    plt.ylabel('Variância Explicada Acumulada')
    plt.title('Variância Explicada Acumulada com PCA')
    plt.grid(True)
    plt.show()

# Visualizar a variância explicada acumulada
pca_plot(cumulative_explained_variance, scaled_data)

### PCA para melhor número de colunas

In [114]:
# # Definir o limiar de variância desejada
# desired_explained_variance = 0.95

# # Selecionar o número de componentes com base no limiar de variância explicada
# num_components = np.argmax(cumulative_explained_variance >= desired_explained_variance) + 1

# # Exibir o número selecionado de componentes
# print(f"Número de componentes para explicar {desired_explained_variance:.2f} da variância: {num_components}")

# # Ajustar o PCA com o número selecionado de componentes
# pca = PCA(n_components=num_components)
# PCA_ds = pd.DataFrame(pca.fit_transform(scaled_data), columns=[f"PC{i+1}" for i in range(num_components)])

# # Exibir as primeiras linhas do DataFrame com os componentes principais
# print("DataFrame com os componentes principais:")
# print(PCA_ds)

### PCA para duas colunas

In [ ]:
# Ajustar o PCA com o número selecionado de componentes
pca = PCA(n_components=2)
PCA_ds = pd.DataFrame(pca.fit_transform(scaled_data))

# Exibir as primeiras linhas do DataFrame com os componentes principais
print("DataFrame com os componentes principais:")
print(PCA_ds)

In [ ]:
PCA_ds

## Verificando pelo Método Silhouette 

In [ ]:
# Definir o intervalo de valores de K para explorar
k_values = range(2, 10)
silhouette_scores = []

best_k = None
best_silhouette_score = -1

# Iterar através dos diferentes valores de K
for k in k_values:
    km = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=100, random_state=42)
    km.fit(PCA_ds)
    
    # Calcular a média da pontuação de Silhouette
    silhouette_avg = silhouette_score(PCA_ds, km.labels_)
    silhouette_scores.append(silhouette_avg)
    
    # Atualizar o melhor valor de K se uma pontuação de Silhouette mais alta for encontrada
    if silhouette_avg > best_silhouette_score:
        best_k = k
        best_silhouette_score = silhouette_avg

# Exibir o melhor valor de K e a pontuação de Silhouette correspondente
print(f"Melhor valor de K: {best_k}")
print(f"Melhor Pontuação de Silhouette: {best_silhouette_score:.4f}")

plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, marker='o', linestyle='-', color='b')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Pontuação de Silhouette')
plt.title('Pontuação de Silhouette para Diferentes Valores de K')
plt.grid(True)
plt.show()

In [ ]:
# Definir o intervalo de k_values
k_values = range(2, 8)  # Começa em 2 porque o Silhouette Score não é definido para k=1

# Criar a figura e os subplots
fig, axs = plt.subplots(2, 3, figsize=(18, 10))  # Ajuste o tamanho da figura conforme necessário
fig.suptitle('Método Silhouette para Encontrar o k Ideal por Nível', fontsize=16)

# Definir a paleta de cores
viridis_palette = sns.color_palette("viridis", n_colors=6)

# Iterar sobre cada nível e plotar em subplots
for nivel in range(1, 7):
    # Filtrar o DataFrame dentro do nível 1
    df_nivel = df.loc[df['Categoria Sinistro'] == nivel]
    
    silhouette_scores = []

    # Calcular o Silhouette Score para cada valor de k
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(df_nivel)
        silhouette_avg = silhouette_score(df_nivel, cluster_labels)
        silhouette_scores.append(silhouette_avg)

    # Escolher o subplot para o nível atual
    ax = axs[(nivel - 1) // 3, (nivel - 1) % 3]
    sns.lineplot(x=k_values, y=silhouette_scores, marker='o', linewidth=2, color=viridis_palette[nivel - 1], ax=ax)
    ax.set_xlabel('Número de Clusters (k)')
    ax.set_ylabel('Silhouette Score')
    ax.set_title(f'Nível {nivel}')
    ax.grid(True, linestyle='--', alpha=0.7)

# Ajustar o layout para que não haja sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta o título da figura
plt.show()

In [ ]:
# Definir o intervalo de k_values
k_values = range(2, 8)  # Começa em 2 porque o Silhouette Score não é definido para k=1

# Criar a figura e os subplots
fig, axs = plt.subplots(2, 3, figsize=(18, 10))  # Ajuste o tamanho da figura conforme necessário
fig.suptitle('Método Silhouette para Encontrar o k Ideal por Nível', fontsize=16)

# Iterar sobre cada nível e plotar em subplots
for nivel in range(1, 7):
    # Filtrar o DataFrame para o nível atual
    df_nivel = df.loc[df['Categoria Sinistro'] == nivel].copy()
    
    silhouette_scores = []

    # Calcular o Silhouette Score para cada valor de k
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(df_nivel)
        silhouette_avg = silhouette_score(df_nivel, cluster_labels)
        silhouette_scores.append(silhouette_avg)

    # Escolher uma paleta de cores
    viridis_palette = sns.color_palette("viridis", n_colors=6)

    # Escolher o subplot para o nível atual
    ax = axs[(nivel - 1) // 3, (nivel - 1) % 3]
    sns.lineplot(x=k_values, y=silhouette_scores, marker='o', linewidth=2, color=viridis_palette[nivel - 1], ax=ax)
    ax.set_xlabel('Número de Clusters (k)')
    ax.set_ylabel('Silhouette Score')
    ax.set_title(f'Nível {nivel}')
    ax.grid(True, linestyle='--', alpha=0.7)

# Ajustar o layout para que não haja sobreposição
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Ajusta o título da figura
plt.show()


---

In [ ]:
X = df.values

range_n_clusters = [6, 7, 8, 9, 10]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    ax1.set_xlim([-0.1, 1])
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator seed of 10 for reproducibility
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    centers = clusterer.cluster_centers_
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d" % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()


---

In [ ]:
# Definir o intervalo de k_values e a porcentagem de dados de teste
k_values = range(2, 9)  # k variando de 3 até 8
test_size = 0.3
random_state = 42

# Iterar sobre os níveis de 'Categoria Sinistro'
for nivel in range(1, 7):
    # Filtrar o DataFrame para o nível atual
    df_nivel = df.loc[df['Categoria Sinistro'] == nivel]
    
    # Dividir os dados entre treino e teste
    dados_treino, dados_teste = train_test_split(df_nivel, test_size=test_size, random_state=random_state)
    
    # Inicializar o StandardScaler e normalizar os dados de treino e teste
    scaler = StandardScaler()
    dados_treino_normalizado = scaler.fit_transform(dados_treino)
    dados_teste_normalizado = scaler.transform(dados_teste)

    # Iterar sobre os diferentes valores de k
    for k in k_values:
        # Aplicar o K-Means no conjunto de treino
        kmeans = KMeans(n_clusters=k, random_state=random_state)
        kmeans.fit(dados_treino_normalizado)
        
        # Prever os clusters para o conjunto de teste
        clusters_teste = kmeans.predict(dados_teste_normalizado)
        
        # Calcular o Silhouette Score para o conjunto de teste
        silhouette_avg = silhouette_score(dados_teste_normalizado, clusters_teste)
        
        # Imprimir o Silhouette Score para o nível atual e valor de k
        print(f"Nível {nivel} - k={k} - Silhouette Score para os dados de teste: {silhouette_avg:.4f}")
    print("-" * 50)  # Separador para cada nível

In [ ]:
# Definir a porcentagem de dados de teste e outros parâmetros
test_size = 0.3
random_state = 42
k = 5  # Pode ser ajustado para o valor de k desejado

# Criar a figura com subplots para os 6 níveis
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(18, 12))  # 2 linhas e 3 colunas de subplots
fig.suptitle("Clusters previstos pelo K-Means para cada nível", fontsize=16)

# Iterar sobre os níveis de 'Categoria Sinistro'
for nivel in range(1, 7):
    # Filtrar o DataFrame para o nível atual
    df_nivel = df.loc[df['Categoria Sinistro'] == nivel]
    
    # Dividir os dados entre treino e teste
    dados_treino, dados_teste = train_test_split(df_nivel, test_size=test_size, random_state=random_state)
    
    # Normalizar os dados de treino e teste
    scaler = StandardScaler()
    dados_treino_normalizado = scaler.fit_transform(dados_treino)
    dados_teste_normalizado = scaler.transform(dados_teste)
    
    # Aplicar o K-Means no conjunto de treino
    kmeans = KMeans(n_clusters=k, random_state=random_state)
    kmeans.fit(dados_treino_normalizado)
    
    # Prever os clusters para o conjunto de teste
    clusters_teste = kmeans.predict(dados_teste_normalizado)
    
    # Aplicar PCA para reduzir a dimensionalidade para 2 componentes
    pca = PCA(n_components=2)
    dados_teste_pca = pca.fit_transform(dados_teste_normalizado)
    
    # Selecionar o subplot correspondente ao nível atual
    ax = axs[(nivel - 1) // 3, (nivel - 1) % 3]
    
    # Plotar os clusters no espaço PCA
    scatter = ax.scatter(dados_teste_pca[:, 0], dados_teste_pca[:, 1], c=clusters_teste, cmap='viridis', s=50)
    
    ax.set_title(f"Nível {nivel}")
    ax.set_xlabel("Componente Principal 1")
    ax.set_ylabel("Componente Principal 2")

# Ajustar o layout para que os gráficos não se sobreponham
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Adicionar uma barra de cores para os clusters
cbar = fig.colorbar(scatter, ax=axs, orientation='vertical', fraction=0.02, pad=0.02)
cbar.set_label('Clusters')

plt.show()

In [ ]:
print(dados_teste_pca[0])

In [ ]:
# Definir a porcentagem de dados de teste e o random_state
test_size = 0.3
random_state = 42

# Iterar sobre os níveis de 'Categoria Sinistro'
for nivel in range(1, 7):
    # Filtrar o DataFrame para o nível atual
    df_nivel = df.loc[df['Categoria Sinistro'] == nivel]
    
    # Dividir os dados entre treino e teste
    dados_treino, dados_teste = train_test_split(df_nivel, test_size=test_size, random_state=random_state)
    
    # Inicializar o StandardScaler e normalizar os dados de treino e teste
    scaler = StandardScaler()
    dados_treino_normalizado = scaler.fit_transform(dados_treino)
    dados_teste_normalizado = scaler.transform(dados_teste)
    
    # Aplicar K-Means
    k = 5  # Definir o número de clusters (ou variar conforme desejado)
    kmeans = KMeans(n_clusters=k, random_state=random_state)
    kmeans.fit(dados_treino_normalizado)
    clusters_teste = kmeans.predict(dados_teste_normalizado)
    
    # Aplicar PCA para reduzir para 3 componentes
    pca = PCA(n_components=3)
    dados_teste_pca = pca.fit_transform(dados_teste_normalizado)
    
    # Plotar os clusters em 3D com a projeção do PCA
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Scatter plot com os clusters usando a paleta Viridis
    scatter = ax.scatter(dados_teste_pca[:, 0], dados_teste_pca[:, 1], dados_teste_pca[:, 2], 
                         c=clusters_teste, cmap='viridis', s=50, alpha=0.8)
    
    ax.set_title(f"Clusters previstos pelo K-Means (3D PCA) - Nível {nivel}")
    ax.set_xlabel("Componente Principal 1")
    ax.set_ylabel("Componente Principal 2")
    ax.set_zlabel("Componente Principal 3")
    
    # Adicionar legenda para os clusters
    legend = ax.legend(*scatter.legend_elements(), title="Clusters")
    ax.add_artist(legend)
    
    plt.show()